# CS838 Project Stage3

Yuying Chen
Maria Kamenetsky

In [82]:
#!which -a python
#!echo $PATH

In [1]:
#import modules
import csv
import py_entitymatching as em
import os
import sys
import re
import time
import shutil
import collections
import pandas as pd
from sklearn import tree
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
import scipy
import matplotlib
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import linear_model
from sklearn.ensemble import ExtraTreesClassifier
from sklearn import svm
from sklearn.model_selection import KFold
import random

We first set our goals: we want to create a matcher that achieves precsion of at least 90% and recall as high as possible. In our team, we then went through and explored the idea and defined what we would consider to be a match and not a match. We elaborate below

## Read input tables

Since the original two movie dataset that we selected don't have too many overlaps, we changed one of them to another movie dataset. The first dataset is from kaggle.com. The second dataset is from movielens.org.

We added an id column for both datasets.

In [2]:
A = em.read_csv_metadata('../../DataforProject/movie1_stage3.csv', key='id',encoding='mac_roman')
B = em.read_csv_metadata('../../DataforProject/movie2_stage3.csv', key='movieId',encoding='mac_roman')

Metadata file is not present in the given path; proceeding to read the csv file.
Metadata file is not present in the given path; proceeding to read the csv file.


In [3]:
print('Number of tuples in A: ' + str(len(A)))
print('Number of tuples in B: ' + str(len(B)))
print('Number of tuples in A x B: ' + str(len(A)*len(B)))

Number of tuples in A: 5043
Number of tuples in B: 27278
Number of tuples in A x B: 137562954


In [4]:
A.head() 

,id,title,genres,director_name,plot_keywords,actor_3_facebook_likes,cast_total_facebook_likes,content_rating,budget,duration,...,actor_2_facebook_likes,imdb_score,movie_facebook_likes,language,actor_1_facebook_likes,facenumber_in_poster,actor_1_name,num_user_for_reviews,aspect_ratio,num_critic_for_reviews
0,1,avatar,action|adventure|fantasy|sci-fi,james cameron,avatar|future|marine|native|paraplegic,855,4834,pg-13,237000000,178,...,936,7.9,33000,english,1000,0,cch pounder,3054,1.78,723
1,2,pirates of the caribbean: at world's end,action|adventure|fantasy,gore verbinski,goddess|marriage ceremony|marriage proposal|pirate|singapore,1000,48350,pg-13,300000000,169,...,5000,7.1,0,english,40000,0,johnny depp,1238,2.35,302
2,3,spectre,action|adventure|thriller,sam mendes,bomb|espionage|sequel|spy|terrorist,161,11700,pg-13,245000000,148,...,393,6.8,85000,english,11000,1,christoph waltz,994,2.35,602
3,4,the dark knight rises,action|thriller,christopher nolan,deception|imprisonment|lawlessness|police officer|terrorist plot,23000,106759,pg-13,250000000,164,...,23000,8.5,164000,english,27000,0,tom hardy,2701,2.35,813
4,5,star wars: episode vii - the force awakens,documentary,doug walker,.,.,143,.,.,.,...,12,7.1,0,.,131,0,doug walker,.,.,.


In [5]:
B.head()

,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II,Comedy,1995


In [6]:
em.get_key(A), em.get_key(B)

('id', 'movieId')

Since the tables are very large let's downsample:

In [7]:
A1, B1 = em.down_sample(A,B, 200,1, show_progress=True)
len(A1), len(B1)

0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:00:00


(176, 200)

## Block Tables To Get Candidate Set

Block on attribute 'year' using an overlap blocker since the same movie must have the same year. The output attributes of table A is limited to id, title, year and genres to match the information provided by table B.

In [8]:
# Create overlap blocker
ob = em.OverlapBlocker()

C = ob.block_tables(A, B, 'year', 'year',
                   l_output_attrs=['id', 'title', 'year', 'genres'],
                   r_output_attrs=['movieId', 'title','year', 'genres'],
                   overlap_size=1, show_progress=True)
em.to_csv_metadata(C, './candidate_pairs.csv') #I really like this function

0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:00:16
File already exists at ./candidate_pairs.csv; Overwriting it
Metadata file already exists at ./candidate_pairs.metadata. Overwriting it


True

In [9]:
len(C)

3288743

## Match tuple pairs in candidate set

### Import Pre-Determined Golden Data

We randomly sampled 500 tuple pairs for labeling. There are 201 matches and 299 mismatches. **gold.csv** is our gold dataset of perfect matches that we have determined prior to this. 

In [92]:
#G = pd.read_csv("../../DataforProject/gold.csv", encoding='mac_roman')
#len(G)

In [10]:
G = em.read_csv_metadata('../../DataforProject/gold.csv', 
                         key='_id',
                            ltable=A, rtable=B, 
                        fk_ltable='ltable_id', fk_rtable='rtable_movieId',
                         encoding='mac_roman')
len(G)

Metadata file is not present in the given path; proceeding to read the csv file.


500

In [11]:
G.head(10)

,_id,ltable_id,rtable_movieId,ltable_movie_title,ltable_title_year,ltable_genres,rtable_title,rtable_year,rtable_genres,label
0,2433,894,64716,seven pounds,2008,drama|romance,Seven Pounds,2008,Drama,1
1,4545,2842,4015,"dude, where's my car?",2000,comedy|mystery,"Dude, Where's My Car?",2000,Comedy|Sci-Fi,1
2,6907,3697,104908,insidious: chapter 2,2013,fantasy|horror|thriller,Insidious: Chapter 2,2013,Horror|Thriller,1
3,24168,4384,2891,"happy, texas",1999,comedy|crime|romance,"Happy, Texas",1999,Comedy,1
4,31352,1844,7980,a bridge too far,1977,drama|history|war,"Bridge Too Far, A",1977,Action|Drama|War,1
5,35987,874,2676,instinct,1999,drama|thriller,Instinct,1999,Drama|Thriller,1
6,37526,2582,1599,steel,1997,action|crime|sci-fi,Steel,1997,Action,1
7,50124,1212,1408,the last of the mohicans,1992,action|adventure|drama|romance|war,"Last of the Mohicans, The",1992,Action|Romance|War|Western,1
8,57296,2987,66200,two lovers,2008,drama|romance,Two Lovers,2008,Drama|Romance,1
9,64913,2952,2841,stir of echoes,1999,horror|mystery|thriller,Stir of Echoes,1999,Horror|Mystery|Thriller,1


In [95]:
#G = em.label_table(G, 'gold')

We note here, that based on what we saw in the creation of the golden data, we establish a *correct* match as a perfect match on year and title, where title is lower case. For example, we would not consider tuple *(insidious, 2013)* and tuple *(insidious: chapter 2, 2013)* to be a match. We do so because we cannot reliably say that the sequal refers to the original. 

### Splitting Labeled Data into Development and Evaluation Set

We next want to split our data into a development set, where we will iteratively train and test various classifiers on the data. The evaluation set will be our final hold out set which we will test last and only once, so that we avoid overfitting. 

Let *I* be the development set and *J* be the evaluation set. We split the data into a 70-30 split for development-evaluation.

In [12]:
IJ = em.split_train_test(G, train_proportion=0.7, random_state=0)
I = IJ['train']
J = IJ['test']

**Package note**: the above split_train_test did not work if I imported G using pandas read_csv, even with encoding. I had to import it using
    ```em.read_csv_metadata()```, which is a little confusing to first-time user since it's a csv, but not necessarily metadata.
    Is there another function that would import metadata and do something with it separately? Or a way to interface with ```pd.read_csv()``` so that  you can read in a csv, clean it and work with it, and then reimport with ```em.read_csv_metadata()``` somehow?

### Selecting Best Learning-Based Matcher

We first go through and set our various machine-learning matchers that we will cycle through:

In [13]:
# Create a set of ML-matchers
dt = em.DTMatcher(name='DecisionTree', random_state=0)
svm = em.SVMMatcher(name='SVM', random_state=0)
rf = em.RFMatcher(name='RF', random_state=0)
lg = em.LogRegMatcher(name='LogReg', random_state=0)
ln = em.LinRegMatcher(name='LinReg')
nb = em.NBMatcher(name='NaiveBayes')

In exploring the **py_entitymatching**, we first start with using the automatically generated features.

In [14]:
#get the auto features
feature_table = em.get_features_for_matching(A,B)

feature_table.feature_name

0        title_title_jac_qgm_3_qgm_3
1    title_title_cos_dlm_dc0_dlm_dc0
2    title_title_jac_dlm_dc0_dlm_dc0
3                    title_title_mel
4               title_title_lev_dist
5                title_title_lev_sim
6                    title_title_nmw
7                     title_title_sw
Name: feature_name, dtype: object

**Package Note**: The ```get_features_for_matching()``` doesn't work if A and B do not have any common features. Which totally makes sense, but the error message was not clear. Also, it would be nice if for features that are quantitative (for example, length of token), you could also get average length of token, median, sd, based on some sort of blocking scheme. Would this help increase P/R?

In [15]:
#convert to feature vectors
H = em.extract_feature_vecs(I,
                           feature_table=feature_table,
                           attrs_after='label',
                           show_progress=True)

0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:00:01


In [37]:
H.head()

,_id,ltable_id,rtable_movieId,title_title_jac_qgm_3_qgm_3,title_title_cos_dlm_dc0_dlm_dc0,title_title_jac_dlm_dc0_dlm_dc0,title_title_mel,title_title_lev_dist,title_title_lev_sim,title_title_nmw,title_title_sw,label
141,640561,3382,69253,0.368421,0.333333,0.2,0.878788,2,0.818182,9.0,9.0,1
383,1475,2654,93492,0.000000,0.000000,0.0,0.533333,9,0.100000,-4.0,2.0,0
135,620734,4840,2127,0.314286,0.000000,0.0,0.878788,4,0.818182,18.0,18.0,1
493,2413,117,64716,0.000000,0.000000,0.0,0.484127,10,0.166667,-3.0,1.0,0
122,573655,716,112556,0.294118,0.000000,0.0,0.851852,2,0.777778,7.0,7.0,1


### Selecting best ML Matcher

Now we want to select the best matcher. We use k-fold cross validation. We will use 10 folds here and use precision, recall, and F1 to select the best matcher.



In [29]:
#select best ML matcher based on cross-validation
##PRECISION

result = em.select_matcher([dt, rf, svm, ln, lg, nb], table=H,
                          exclude_attrs=['_id', 'ltable_id','rtable_movieId', 'label'],
                          k = 10,
                          target_attr='label', metric='precision', random_state=0)

result['cv_stats']

,Name,Matcher,Num folds,Fold 1,Fold 2,Fold 3,Fold 4,Fold 5,Fold 6,Fold 7,Fold 8,Fold 9,Fold 10,Mean score
0,DecisionTree,<py_entitymatching.matcher.dtmatcher.DTMatcher object at 0x000002D017A2DAC8>,10,1.0,1.0,1.0,1.0,0.928571,1.0,1.0,1.0,1.0,1.0,0.992857
1,RF,<py_entitymatching.matcher.rfmatcher.RFMatcher object at 0x000002D017A2D630>,10,1.0,1.0,1.0,1.0,0.928571,1.0,1.0,1.0,1.0,1.0,0.992857
2,SVM,<py_entitymatching.matcher.svmmatcher.SVMMatcher object at 0x000002D017A2D390>,10,1.0,1.0,1.0,1.0,0.857143,1.0,1.0,1.0,1.0,1.0,0.985714
3,LinReg,<py_entitymatching.matcher.linregmatcher.LinRegMatcher object at 0x000002D017A2D780>,10,1.0,1.0,1.0,1.0,1.000000,1.0,1.0,1.0,1.0,1.0,1.000000
4,LogReg,<py_entitymatching.matcher.logregmatcher.LogRegMatcher object at 0x000002D017A2D940>,10,1.0,1.0,1.0,1.0,0.923077,1.0,1.0,1.0,1.0,1.0,0.992308
5,NaiveBayes,<py_entitymatching.matcher.nbmatcher.NBMatcher object at 0x000002D017A2D6A0>,10,1.0,1.0,1.0,1.0,0.923077,1.0,1.0,1.0,1.0,1.0,0.992308


Based on average precision across the 10 folds, we see that linear regression performs the best, follwed by decision tree and random forest performing equally well. 

**Package Note**: I really like the way these results are displayed - very clear and useful!

In [27]:
#select best ML matcher based on cross-validation
##RECALL

result = em.select_matcher([dt, rf, svm, ln, lg, nb], table=H,
                          exclude_attrs=['_id', 'ltable_id','rtable_movieId', 'label'],
                          k = 5,
                          target_attr='label', metric='recall', random_state=0)

result['cv_stats']

,Name,Matcher,Num folds,Fold 1,Fold 2,Fold 3,Fold 4,Fold 5,Mean score
0,DecisionTree,<py_entitymatching.matcher.dtmatcher.DTMatcher object at 0x000002D017A2DAC8>,5,0.964286,0.966667,1.000000,0.962963,1.000000,0.978783
1,RF,<py_entitymatching.matcher.rfmatcher.RFMatcher object at 0x000002D017A2D630>,5,0.964286,1.000000,0.966667,0.962963,1.000000,0.978783
2,SVM,<py_entitymatching.matcher.svmmatcher.SVMMatcher object at 0x000002D017A2D390>,5,0.928571,0.933333,0.966667,0.925926,0.862069,0.923313
3,LinReg,<py_entitymatching.matcher.linregmatcher.LinRegMatcher object at 0x000002D017A2D780>,5,0.964286,0.933333,0.933333,0.962963,1.000000,0.958783
4,LogReg,<py_entitymatching.matcher.logregmatcher.LogRegMatcher object at 0x000002D017A2D940>,5,0.964286,1.000000,0.966667,1.000000,1.000000,0.986190
5,NaiveBayes,<py_entitymatching.matcher.nbmatcher.NBMatcher object at 0x000002D017A2D6A0>,5,0.964286,0.966667,0.966667,1.000000,1.000000,0.979524


Based on the average recall across the 10 folds, we find that that the logistic regression performs the best, followed by Naive Bayes and than random forest/decision tree. 

For the purposes of our project, the goal is to create a matcher that achieves

**Package note**: Is there a way to extract recall at a K, which the user can specify? For example, of the top 100 tuples that are declared matches, how many of these are true positives.

### Evaluating the Matching Output

First, we want to convert the feature vectors and create a new set _L_ based on the test set _J_, which will have the feature vectors from a feature table

In [39]:
L = em.extract_feature_vecs(J, feature_table=feature_table,
                           attrs_after='label',
                           show_progress=False)

Next, we will trian the matched based on the feature vectors from the development set. Based on the results of the development set, we want to find a matcher that achieves precision of at least 90% and recall as high possible. Based on the nature of our project (matching movie-year across two data sources), we want to minimize the number of false negatives (minimize type 2 error). Therefore, we select the logistic regression as our matcher.

In [48]:
#Train using feature vectors from I

lg.fit(table=H,
      exclude_attrs=['_id', 'ltable_id', 'rtable_movieId', 'label'],
      target_attr='label')

Based on the above, we now want to predict:

In [50]:
L.head()


,_id,ltable_id,rtable_movieId,title_title_jac_qgm_3_qgm_3,title_title_cos_dlm_dc0_dlm_dc0,title_title_jac_dlm_dc0_dlm_dc0,title_title_mel,title_title_lev_dist,title_title_lev_sim,title_title_nmw,title_title_sw,label
90,457561,4831,6881,0.478261,0.333333,0.2,0.911111,2,0.866667,13.0,13.0,1
254,581,4992,39183,0.000000,0.000000,0.0,0.419103,17,0.105263,0.0,3.0,0
283,771,1834,31856,0.000000,0.000000,0.0,0.481624,35,0.125000,-22.0,2.0,0
445,2013,2378,101715,0.000000,0.000000,0.0,0.351852,8,0.111111,-2.0,1.0,0
461,2169,1651,5957,0.000000,0.000000,0.0,0.384722,14,0.125000,-4.0,1.0,0


In [51]:
predictions = lg.predict(table=L, exclude_attrs=['_id', 'ltable_id', 'rtable_movieId', 'label'], 
              append=True, target_attr='predicted', inplace=False)

In [52]:
eval_result = em.eval_matches(predictions, 'label', 'predicted')
em.print_eval_summary(eval_result)

Precision : 96.55% (56/58)
Recall : 100.0% (56/56)
F1 : 98.25%
False positives : 2 (out of 58 positive predictions)
False negatives : 0 (out of 92 negative predictions)


**Package note**: this summary statistics are AWESOME.

From the above test set, we can see that we have minimized false negatives (type 2 error) and still achieved precision of over 90%. 